<a href="https://colab.research.google.com/github/SEXYLIULIU/Assessment3/blob/main/assessmeny3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Required libraries
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, SimpleRNN, Dropout
import matplotlib.pyplot as plt


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!ls "/content/drive/My Drive"


 100OLYMP.tar.bz2	       'archive (2).zip'  'Concepts of AI'
'A1_T3_While_Loops (1).ipynb'   Assignment3.gdoc  'Undergrad assignment 3.drawio'
'A1_T3_While_Loops (2).ipynb'  'Colab Notebooks'


In [7]:
zip_path = "/content/drive/My Drive/archive (2).zip"


In [8]:
import zipfile

extracted_dir = "/content/extracted_data"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)
print(f"Files extracted to: {extracted_dir}")


Files extracted to: /content/extracted_data


In [14]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU
import matplotlib.pyplot as plt

In [16]:
def get_supported_files(directory, extensions=('.csv', '.json')):
    supported_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(extensions):
                supported_files.append(os.path.join(root, file))
    return supported_files

# Define the extracted directory
extracted_dir = "/content/extracted_data"
files = get_supported_files(extracted_dir)
print(f"Found {len(files)} supported files.")


Found 9210 supported files.


In [17]:
def load_and_combine(files):
    combined_data = []
    for file in files:
        try:
            if file.endswith('.csv'):
                data = pd.read_csv(file)
            elif file.endswith('.json'):
                data = pd.read_json(file)
            else:
                continue

            # Check for required columns
            if set(['Date', 'Open', 'High', 'Low', 'Close', 'Volume']).issubset(data.columns):
                data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
                combined_data.append(data)
            else:
                print(f"Skipping file {file}: Required columns not found.")
        except Exception as e:
            print(f"Error processing file {file}: {e}")

    # Combine all data into a single DataFrame
    return pd.concat(combined_data, ignore_index=True) if combined_data else pd.DataFrame()

# Load and combine data
combined_data = load_and_combine(files)

# Check the combined dataset
print(f"Combined Data Shape: {combined_data.shape}")
print(combined_data.head())


Streaming output truncated to the last 5000 lines.
Skipping file /content/extracted_data/stock_market_data/sp500/json/PNW.json: Required columns not found.
Skipping file /content/extracted_data/stock_market_data/sp500/json/MAA.json: Required columns not found.
Skipping file /content/extracted_data/stock_market_data/sp500/json/MHK.json: Required columns not found.
Skipping file /content/extracted_data/stock_market_data/sp500/json/PH.json: Required columns not found.
Skipping file /content/extracted_data/stock_market_data/sp500/json/YUM.json: Required columns not found.
Skipping file /content/extracted_data/stock_market_data/sp500/json/WST.json: Required columns not found.
Skipping file /content/extracted_data/stock_market_data/sp500/json/NVR.json: Required columns not found.
Skipping file /content/extracted_data/stock_market_data/sp500/json/NI.json: Required columns not found.
Skipping file /content/extracted_data/stock_market_data/sp500/json/EMN.json: Required columns not found.
Skippi

In [3]:
# Convert Date to datetime and sort by date
combined_data['Date'] = pd.to_datetime(combined_data['Date'])
combined_data.sort_values('Date', inplace=True)

# Fill missing values
combined_data.fillna(method='ffill', inplace=True)

# Normalize numerical features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(combined_data[['Open', 'High', 'Low', 'Close', 'Volume']])

# Replace columns with scaled data
combined_data[['Open', 'High', 'Low', 'Close', 'Volume']] = scaled_data

print(combined_data.head())


NameError: name 'pd' is not defined

In [ ]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length, :-1])  # All columns except the last one
        y.append(data[i + seq_length, -1])    # Target: Close price
    return np.array(X), np.array(y)

# Sequence length
sequence_length = 30

# Split the data into training, validation, and test sets
train_size = int(len(scaled_data) * 0.7)
val_size = int(len(scaled_data) * 0.15)

train_data = scaled_data[:train_size]
val_data = scaled_data[train_size:train_size + val_size]
test_data = scaled_data[train_size + val_size:]

# Create sequences
X_train, y_train = create_sequences(train_data, sequence_length)
X_val, y_val = create_sequences(val_data, sequence_length)
X_test, y_test = create_sequences(test_data, sequence_length)

# Print shapes
print(f"Training Data: X={X_train.shape}, y={y_train.shape}")
print(f"Validation Data: X={X_val.shape}, y={y_val.shape}")
print(f"Test Data: X={X_test.shape}, y={y_test.shape}")


In [ ]:
def build_model(model_type, input_shape):
    model = Sequential()
    if model_type == "VanillaRNN":
        model.add(SimpleRNN(50, activation='tanh', input_shape=input_shape))
    elif model_type == "LSTM":
        model.add(LSTM(50, activation='tanh', input_shape=input_shape))
    elif model_type == "GRU":
        model.add(GRU(50, activation='tanh', input_shape=input_shape))
    else:
        raise ValueError("Invalid model type! Choose from 'VanillaRNN', 'LSTM', 'GRU'.")

    model.add(Dense(1))  # Output layer
    model.compile(optimizer='adam', loss='mse')
    return model


In [ ]:
results = {}

for model_type in ["VanillaRNN", "LSTM", "GRU"]:
    print(f"Training {model_type}...")
    model = build_model(model_type, X_train.shape[1:])

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        verbose=1
    )

    # Evaluate on test set
    test_loss = model.evaluate(X_test, y_test, verbose=0)
    results[model_type] = test_loss
    print(f"{model_type} Test Loss: {test_loss}")


In [ ]:
# Predict
predictions = model.predict(X_test)

# Reverse scaling
y_test_actual = scaler.inverse_transform(
    np.concatenate((np.zeros((y_test.shape[0], scaled_data.shape[1] - 1)), y_test.reshape(-1, 1)), axis=1)
)[:, -1]

predictions_actual = scaler.inverse_transform(
    np.concatenate((np.zeros((predictions.shape[0], scaled_data.shape[1] - 1)), predictions), axis=1)
)[:, -1]

# Plot
plt.figure(figsize=(10, 6))
plt.plot(y_test_actual, label='Actual Prices')
plt.plot(predictions_actual, label='Predicted Prices')
plt.title("Actual vs Predicted Prices")
plt.xlabel("Time")
plt.ylabel("Price")
plt.legend()
plt.show()
